### Writing the utility python file

In [1]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import gc
import re

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df, table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_', regex = True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(), table_config['columns']))
    expected_col.sort()
    df.columns = list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis = 1)
    if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file", mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded", missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


### Writing the utility YAML file

In [2]:
%%writefile file.yaml
file_type: csv
dataset_name: df_test
file_name: survival_data
inbound_delimiter: ","
columns:
    - Age_start_observed
    - Age_end
    - Truncated
    - Censored
    - Dead
    - Date_start_observed
    - Date_end_observed

Overwriting file.yaml


In [3]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [4]:
# Show the config dictionary
config_data

{'file_type': 'csv',
 'dataset_name': 'df_test',
 'file_name': 'survival_data',
 'inbound_delimiter': ',',
 'columns': ['Age_start_observed',
  'Age_end',
  'Truncated',
  'Censored',
  'Dead',
  'Date_start_observed',
  'Date_end_observed']}

In [5]:
# Reading the file using dask
import dask.dataframe as dd
df_sample = dd.read_csv('survival_data.csv')
df_sample.head()

,Unnamed: 0,age_start_observed,age_end,is_truncated,is_censored,is_dead,date_start_observed,date_end_observed
0,15113102,0.000000,9.097335,False,True,False,1908-11-17,1917-12-22
1,41505894,0.000000,64.486689,False,True,False,1828-09-13,1893-03-10
2,24774171,0.000000,33.071552,False,True,False,1911-02-07,1944-03-04
3,97834936,34.834566,68.778258,True,True,False,1820-01-01,1853-12-10
4,45793809,0.000000,95.948358,False,False,True,1870-05-29,1966-05-11


In [6]:
# View the computational efficiency
%time df_sample = dd.read_csv("survival_data.csv")

Wall time: 24.7 ms


In [7]:
# Read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
df_test = dd.read_csv(source_file)
df_test.head()

,Unnamed: 0,age_start_observed,age_end,is_truncated,is_censored,is_dead,date_start_observed,date_end_observed
0,15113102,0.000000,9.097335,False,True,False,1908-11-17,1917-12-22
1,41505894,0.000000,64.486689,False,True,False,1828-09-13,1893-03-10
2,24774171,0.000000,33.071552,False,True,False,1911-02-07,1944-03-04
3,97834936,34.834566,68.778258,True,True,False,1820-01-01,1853-12-10
4,45793809,0.000000,95.948358,False,False,True,1870-05-29,1966-05-11


In [8]:
# Making some changes in the dataframe
df = df_test.rename(columns = {"age_start_observed": "Age_start_observed", "age_end": "Age_end", "is_truncated" : "Truncated", "is_censored": "Censored", "is_dead" : "Dead", "date_start_observed" : "Date_start_observed", "date_end_observed" : "Date_end_observed"})
df = df.drop('Unnamed: 0', axis = 1)
df

,Age_start_observed,Age_end,Truncated,Censored,Dead,Date_start_observed,Date_end_observed
npartitions=102,,,,,,,
,float64,float64,bool,bool,bool,object,object
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [9]:
# Always checking for missing values
df.isnull().sum().compute()

Age_start_observed     0
Age_end                0
Truncated              0
Censored               0
Dead                   0
Date_start_observed    0
Date_end_observed      0
dtype: int64

### Something get wrong with the code in the following line

In [10]:
#util.col_header_val(df, config_data)

In [11]:
# Validate number of columns and column names
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['Age_start_observed', 'Age_end', 'Truncated', 'Censored', 'Dead',
       'Date_start_observed', 'Date_end_observed'],
      dtype='object')
columns of YAML are: ['Age_start_observed', 'Age_end', 'Truncated', 'Censored', 'Dead', 'Date_start_observed', 'Date_end_observed']


### As we can see, they have the same amount of columns. The names of them are also the same.

In [12]:
# Create the csv file into text file
import csv
with open('survival_data.txt', "w") as my_output_file:
    with open('survival_data.csv', "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

In [13]:
# Create the text seperated file into gz format
import gzip
import shutil
with open('survival_data.txt', 'rb') as f_in, gzip.open('survival_data.gz', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

In [14]:
df.compute()

,Age_start_observed,Age_end,Truncated,Censored,Dead,Date_start_observed,Date_end_observed
0,0.000000,9.097335,False,True,False,1908-11-17,1917-12-22
1,0.000000,64.486689,False,True,False,1828-09-13,1893-03-10
2,0.000000,33.071552,False,True,False,1911-02-07,1944-03-04
3,34.834566,68.778258,True,True,False,1820-01-01,1853-12-10
4,0.000000,95.948358,False,False,True,1870-05-29,1966-05-11
...,...,...,...,...,...,...,...
837199,8.053810,53.177762,True,True,False,1820-01-01,1865-02-14
837200,0.000000,1.344027,False,True,False,1857-02-09,1858-06-15
837201,0.000000,30.505584,False,False,True,1833-12-24,1864-06-26
837202,0.000000,24.268282,False,False,True,1900-04-27,1924-08-03


### There are 88.809.774 rows and 7 columns in the csv file

In [15]:
import os

def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size
 
file_path = 'survival_data.csv'
size = get_file_size(file_path)
print('File size: '+ str(size) +' bytes')

File size: 6525517581 bytes


### This file is 6.525.517.581 bytes!